# Bir CSV dosyasından içerisinde kahve geçen kelimeleri (ürünleri) bulma

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
import os

In [2]:
pyspark = SparkSession.builder \
.master("local[4]") \
.appName("Create a RDD") \
.config("spark.executor.memory", "4g") \
.config("spark.driver.memory", "2g") \
.getOrCreate()

23/04/14 19:37:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
sc = pyspark.sparkContext
sc.setLogLevel("error")

In [10]:
file = f"{os.getcwd()}/OnlineRetail.csv"

In [33]:
retailRDD = sc.textFile(file)

In [34]:
retailRDD.take(5)

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 '536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom']

### Header'ı Silme

In [1]:
firstline = retailRDD.first()
firstlineRDD = sc.parallelize(firstline)
retailRDD_without_header = retailRDD.subtract(firstlineRDD)
retailRDD_without_header.take(5)

NameError: name 'retailRDD' is not defined

### Birim miktarı 30'dan büyük olanları filtreleme

In [2]:
filteredRDD = retailRDD_without_header.filter(lambda x: int(x.split(";")[3]) > 30)
filteredRDD.take(5)

NameError: name 'retailRDD_without_header' is not defined

### Ürün adında coffee geçenleri filtreleme

In [41]:
coffeeRDD = retailRDD_without_header.filter(lambda x: 'COFFEE' in x.split(";")[2])
coffeeRDD.take(5)

['536787;37370;RETRO COFFEE MUGS ASSORTED;6;2.12.2010 15:24;1,06;17850;United Kingdom',
 '536805;37370;RETRO COFFEE MUGS ASSORTED;12;2.12.2010 16:38;1,25;14775;United Kingdom',
 '536865;37370;RETRO COFFEE MUGS ASSORTED;1;3.12.2010 11:28;16,13;000000;United Kingdom',
 '537231;22304;COFFEE MUG BLUE PAISLEY DESIGN;6;6.12.2010 09:21;2,55;13652;United Kingdom',
 '538035;37370;RETRO COFFEE MUGS ASSORTED;3;9.12.2010 13:03;1,25;16065;United Kingdom']

### Unit > 30 and contains COFFEE

In [49]:
def pair_filter(line: str):
    quantity = int(line.split(";")[3])
    desc = line.split(";")[2]
    
    return (quantity > 30) & ("COFFEE" in desc)

In [50]:
retailRDD_without_header.filter(lambda x: pair_filter(x)).take(10)

['543054;22300;COFFEE MUG DOG + BALL DESIGN;36;3.02.2011 10:39;2,1;13394;United Kingdom',
 '544480;22303;COFFEE MUG APPLES DESIGN;72;21.02.2011 10:32;2,1;14646;Netherlands',
 '544690;22303;COFFEE MUG APPLES DESIGN;36;23.02.2011 08:48;2,1;14156;EIRE',
 '545533;22303;COFFEE MUG APPLES DESIGN;36;3.03.2011 14:04;2,1;12524;Germany',
 '545668;37370;RETRO COFFEE MUGS ASSORTED;144;4.03.2011 17:02;1,06;14031;United Kingdom',
 '550137;21216;SET 3 RETROSPOT TEA,COFFEE,SUGAR;72;14.04.2011 13:59;4,25;12477;Germany',
 '551840;22300;COFFEE MUG DOG + BALL DESIGN;36;4.05.2011 13:36;2,1;15144;United Kingdom',
 '552692;21399;BLUE POLKADOT COFFEE MUG;48;10.05.2011 15:19;0,39;13990;United Kingdom',
 '553546;37370;RETRO COFFEE MUGS ASSORTED;108;17.05.2011 15:42;1,25;12415;Australia',
 '553867;22970;LONDON BUS COFFEE MUG;36;19.05.2011 14:19;2,1;17306;United Kingdom']